In [1]:
import pandas as pd
from Bio import SeqIO, Entrez

In [4]:
Entrez.email = #entrez requires a valid email address for downloading the coding sequences

In [5]:
tabs3xl = pd.ExcelFile('tableS3_spike.xlsx')

In [10]:
#Download coding sequences
for mut in tabs3xl.sheet_names:
    
    print(mut)
    
    mutdf = pd.read_excel(tabs3xl, mut)
    hitacc = list(mutdf.Accession)
    if 'Accession' in hitacc:
        hitacc.remove('Accession')
        
    handle = Entrez.efetch(db="protein", id=list(hitacc), rettype="fasta_cds_na", retmode="text")
    fastaout = handle.read()
    
    with open('hitcds/hitcds_%s.fas'%mut, 'w') as out:
        out.write(fastaout)

    print("Successfully downloaded %i coding sequences for the %i VMF hit proteins\n"%(fastaout.count('>'), len(hitacc)))
    
    

P681R
Successfully downloaded 41 coding sequences for the 38 VMF hit proteins

L452R
Successfully downloaded 2 coding sequences for the 3 VMF hit proteins

T19R
Successfully downloaded 60 coding sequences for the 114 VMF hit proteins

V1176F
Successfully downloaded 6 coding sequences for the 8 VMF hit proteins

T1027I
Successfully downloaded 6 coding sequences for the 8 VMF hit proteins

E484K
Successfully downloaded 5 coding sequences for the 4 VMF hit proteins

K417T
Successfully downloaded 4 coding sequences for the 6 VMF hit proteins

R190S
Successfully downloaded 87 coding sequences for the 93 VMF hit proteins

D138Y
Successfully downloaded 2 coding sequences for the 2 VMF hit proteins

P26S
Successfully downloaded 7 coding sequences for the 9 VMF hit proteins

T20N
Successfully downloaded 862 coding sequences for the 818 VMF hit proteins

L18F
Successfully downloaded 162 coding sequences for the 194 VMF hit proteins

A701V
Successfully downloaded 11 coding sequences for the 12 VM

In [13]:
#load 7 amino acid VMFs for Spike substitutions
spikeHFs = pd.read_excel('tableS2_spike7aa.xlsx')
spikeHFs

,Mutation,7aa
0,A570D,RDIDDTT
1,A67V,WFHVIHV
2,A701V,SLGVENS
3,D1118H,ITTHNTF
4,D138Y,FCNYPFL
5,D215G,LVRGLPQ
6,D405N,IRGNEVR
7,D614G,LYQGVNC
8,D796Y,PIKYFGG
9,D80A,KRFANPV


In [14]:
#parse nucleotides motifs corresponding to each 7aa VMF in dictionary
allcdsmot = {}

for mut,mot in zip(list(spikeHFs.Mutation), list(spikeHFs['7aa'])):
        
    allcdsmot.update({mut:[]})
    
    for rec in SeqIO.parse('hitcds/hitcds_%s.fas'%mut, 'fasta'):
                
        aaseq = rec.seq.translate()
                
        motpoz = str(aaseq).find(mot)
                
        cdsmot = str(rec.seq)[motpoz*3:(motpoz*3)+21]
        
        allcdsmot[mut].append(cdsmot)

#and summarise in csv file
sumlist = []
for mut in list(allcdsmot):
    for nuc in set(allcdsmot[mut]):
        sumlist.append([mut, nuc, allcdsmot[mut].count(nuc), list(spikeHFs[spikeHFs['Mutation'] == mut]['7aa'])[0]])
        
sumdf = pd.DataFrame(sumlist)
sumdf.columns = ['mutation', 'nuc_seq', 'count', '7aa']
sumdf.to_csv('spikemut_7aahit_CDS.csv', index=False)
sumdf        

C:\Users\spyro\anaconda3\lib\site-packages\Bio\Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


,mutation,nuc_seq,count,7aa
0,A570D,CGCGACATCGACGACACCACC,1,RDIDDTT
1,A570D,AGGGATATTGATGACACAACT,1,RDIDDTT
2,A570D,AGAGACATTGATGACACTACT,8,RDIDDTT
3,A570D,CGCGACATCGACGACACCACG,1,RDIDDTT
4,A570D,AGAGATATCGACGACACCACA,1,RDIDDTT
...,...,...,...,...
624,V213G,ATCAATTTGGGCCGTGACCTC,1,INLGRDL
625,V213G,ATAAACTTGGGTAGAGATCTT,1,INLGRDL
626,V213G,ATTAATCTAGGAAGAGATTTA,1,INLGRDL
627,V213G,ATTAACCTTGGTAGGGATCTA,1,INLGRDL


In [20]:
#same as above for the SARS-CoV-2 spike coding sequences
    #only the accessions but not the sequences themselves are provided in this github repo due to GISAID data sharing restrictions

spikemot = {}
for mut,mot in zip(list(spikeHFs.Mutation), list(spikeHFs['7aa'])):
        
    spikemot.update({mut:[]})
    
    for rec in SeqIO.parse('sc2_spikes.fas', 'fasta'):
                
        aaseq = rec.seq.translate()
                
        motpoz = str(aaseq).find(mot)
                
        cdsmot = str(rec.seq)[motpoz*3:(motpoz*3)+21]
        
        spikemot[mut].append(cdsmot)

spikelist = []
for mut in list(spikemot):
    for nuc in set(spikemot[mut]):
        if nuc != '':
            spikelist.append([mut, nuc, spikemot[mut].count(nuc), list(spikeHFs[spikeHFs['Mutation'] == mut]['7aa'])[0]])
        
spikedf = pd.DataFrame(spikelist)
spikedf.columns = ['mutation', 'nuc_seq', 'count', '7aa']
spikedf.to_csv('spikemut_SC2var_CDS.csv', index=False)
spikedf        

,mutation,nuc_seq,count,7aa
0,A570D,AGAGACATTGATGACACTACT,1,RDIDDTT
1,A701V,TCACTTGGTGTAGAAAATTCA,1,SLGVENS
2,D1118H,ATTACTACACACAACACATTT,1,ITTHNTF
3,D138Y,TTTTGTAATTATCCATTTTTG,1,FCNYPFL
4,D215G,TTAGTGCGTGGTCTCCCTCAG,1,LVRGLPQ
5,D405N,ATTAGAGGTAATGAAGTCAGA,1,IRGNEVR
6,D614G,CTTTATCAGGGTGTTAACTGC,9,LYQGVNC
7,D796Y,CCAATTAAATATTTTGGTGGT,5,PIKYFGG
8,D80A,AAGAGGTTTGCTAACCCTGTC,1,KRFANPV
9,E484A,AATGGTGTTGCAGGTTTTAAT,2,NGVAGFN


<br>

Cao *et al.* have searched for VMFs considering only one substitution at a time and disregarding multiple substitutions appearing in a SARS-CoV-2 VOC at the same time. The following substitutions correspond to 7aa VMFs used in the paper that do not actually appear in the Spike protein of any major VOCs because the Spikes have multiple substitutions compared to the ancestral strain within the same 7aa fragment. 

In [21]:
set(spikeHFs.Mutation) - set(spikedf.mutation)

{'A67V',
 'F486V',
 'G496S',
 'N679K',
 'Q498R',
 'R408S',
 'S371F',
 'S371P',
 'S373P',
 'S375F',
 'T20N',
 'T376A'}

<br>

Finally show the intersect (if any) between the nucleotide sequences encoding the 7aa protein motifs in the microbial proteins and in the SARS-CoV-2 VOC Spikes:

In [22]:
for mut in list(spikeHFs.Mutation):

    if set(spikemot[mut]) != {''}:
        print(mut)
        print( set(allcdsmot[mut]).intersection(set(spikemot[mut])) )


A570D
{'AGAGACATTGATGACACTACT'}
A701V
set()
D1118H
set()
D138Y
set()
D215G
set()
D405N
set()
D614G
set()
D796Y
set()
D80A
set()
E484A
set()
E484K
set()
G142D
set()
G339D
set()
G446S
set()
H655Y
set()
K417N
set()
K417T
set()
L452R
set()
L981F
set()
N440K
set()
N501Y
set()
N764K
set()
N856K
set()
N969K
set()
P26S
set()
P681H
set()
P681R
set()
Q493R
set()
Q954H
set()
R190S
set()
S982A
set()
T1027I
set()
T19I
set()
T19R
set()
T478K
set()
T547K
set()
T716I
set()
T95I
set()
V1176F
set()
V213G
set()
Y505H
set()


As indicated above, there is only a single VMF-encoding nucleotide sequence that matches between the microbial protein hits presented by Cao *et al.* and the SARS-CoV-2 Spike coding sequences. 

As shown below there are 8 hits with the matching sequence `AGAGACATTGATGACACTACT`, representing a mere 0.4% of the analysed protein hits.

In [24]:
sumdf[sumdf.nuc_seq=='AGAGACATTGATGACACTACT']

,mutation,nuc_seq,count,7aa
2,A570D,AGAGACATTGATGACACTACT,8,RDIDDTT


In [28]:
8/sum(list(sumdf['count']))

0.004767580452920143